In [1]:
from keras.models import Model
from keras_models.inception_v3 import InceptionV3
from keras.preprocessing import image
import os
import numpy as np
from keras_models.imagenet_utils import preprocess_input
from collections import defaultdict
import json
from tqdm import tqdm
import pickle

Using TensorFlow backend.


In [2]:
base_model = InceptionV3(weights='imagenet')

In [3]:
# print base_model.summary()

In [4]:
model = Model(inputs=[base_model.input], outputs=[base_model.get_layer('avg_pool').output])

In [5]:
root = '../data/food-101/images'
labels = sorted(os.listdir(root))

paths_to_labels = {}

for label in labels:
    label_dir = os.path.join(root, label)
    for name in os.listdir(label_dir):
        if name.endswith('.jpg') or name.endswith('.png') or name.endswith('.gif'):
            path = os.path.join(label_dir, name)
            paths_to_labels[path] = label

categories = defaultdict(list)
for path, label in paths_to_labels.iteritems():
    categories[label].append(path)

print labels

['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheese_plate', 'cheesecake', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_cheese', 'macarons', 'miso_sou

In [6]:
img_path = paths_to_labels.keys()[2]
label = paths_to_labels[img_path]

img = image.load_img(img_path, target_size=(299, 299))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
print preds

[[ 0.21843584  0.79452246  1.55977261 ...,  0.01140913  0.30277196  0.        ]]


In [7]:
print preds

[[ 0.21843584  0.79452246  1.55977261 ...,  0.01140913  0.30277196  0.        ]]


In [8]:
def split_into_batches(items, size=16):
    batches = [[]]
    for item in items:
        if len(batches[-1]) == size:
            batches.append([])
        batches[-1].append(item)
    return batches


In [9]:
dir = '../data/food-101/inception_activations'
try:
    os.mkdir(dir)
except OSError: pass

In [13]:
def make_batch(paths):
    def load_img(path):
        img = image.load_img(path, target_size=(299, 299))
        x = image.img_to_array(img)
        return x
    return preprocess_input(np.stack([load_img(path) for path in paths]))

for label, paths in categories.iteritems():
    out_path = os.path.join(dir, label + '.pickle')
    if os.path.exists(out_path):
        # print 'skipping', label
        continue
    activations = {}
    for path_batch in tqdm(split_into_batches(paths, 16)):
        values = model.predict(make_batch(path_batch))
        for filename, act in zip(path_batch, values):
            activations[filename] = act
    with open(out_path, 'w') as f:
        pickle.dump(activations, f)
    print 'done with', label
    

  0%|          | 0/63 [00:00<?, ?it/s]

done with foie_gras


  0%|          | 0/63 [00:00<?, ?it/s]

done with tacos


  0%|          | 0/63 [00:00<?, ?it/s]

...

done with ceviche


  0%|          | 0/63 [00:00<?, ?it/s]

done with bruschetta


 33%|███▎      | 21/63 [00:15<00:20,  2.02it/s]

KeyboardInterrupt: 